In [1]:
import math
import random
import time
import matplotlib.pyplot as plt 
import datetime
from multiprocessing import Pool
from itertools import combinations, product
from statistics import mean, stdev
from itertools import combinations
from numpy import log, append, setdiff1d, array
from collections import deque # Cola FIFO para la lista tabú
from numpy import argsort # Ordenar lista por indices (GRASP)

from itertools import tee
verbose = False


# Funciones comunes

In [2]:
def leeFichero(nombreFichero ='tsp/a280.tsp'):
    rutaFichero = './tsp/'
    extension = '.tsp'
    fichero = open(rutaFichero + nombreFichero + extension, 'r')
    array_puntos = []

    # Saltamos las cabeceras del fichero #
    for i in range(6):
        fichero.readline()

    for line in fichero:
        try:
            array_temp = line.replace('   ', ' ').replace('  ', ' ').replace('\n', '').split(' ')
            array_temp = [float(item) for item in array_temp if item]

            if(len(array_temp) > 1):
                array_puntos.append(array_temp)
        except:
            pass

    fichero.close()

    return array_puntos

In [3]:
def getDistancia(p1, p2):
    xd = p1[1] - p2[1]
    yd = p1[2] - p2[2]
    
    dij = round(math.sqrt(xd*xd + yd*yd))
    
    return dij

In [4]:
def getMDistancias(nodos):
   
    distancias = [[0]*len(nodos) for i in range(len(nodos))]
    
    for i in range(len(nodos)):
        for j in range(len(nodos)):
            
            if(i==j):
                dij = 0 # Distancia a sí mismo infinita
            else:
                dij = getDistancia(nodos[i], nodos[j])
                
            distancias[i][j] = dij
            
    return distancias

In [5]:
def pintaCamino(camino, nodos, titulo):
    
    x = []
    y = []
    
    for i in range(0, len(camino)-1):
        
        x.append(nodos[camino[i]-1][1])
        y.append(nodos[camino[i]-1][2])
        
    plt.figure(figsize=(12,12))    
    plt.title(titulo)
    plt.scatter(x, y, color='green') 
    
    for i in range(len(x)):
        if(i < len(x)-1):
            plt.arrow(x[i], y[i], x[i+1] - x[i], y[i+1] - y[i], head_width=1, length_includes_head=True)
        else:
             plt.arrow(x[i], y[i], x[0] - x[i], y[0] - y[i], head_width=1, length_includes_head=True)
                
    # Punto rojo inicial
    plt.plot(x[0],y[0], marker='o',
     markerfacecolor='red', markersize=12)
                

In [6]:
def getCosteCamino(camino, distancias, verbose=False):
    coste = 0
    
    for i in range(0, len(camino)-1):
        dij = distancias[camino[i]][camino[i+1]]
        coste += dij # Sumamos la distancia de un nodo al siguiente
        if(verbose):
            print(f"Acumulando Distancia {camino[i]} -> {camino[i+1]} = {dij}")
        
        
    dij = distancias[camino[-1]][camino[0]] #Sumamos la distancia del ultimo al primero
    coste += dij
    if(verbose):
            print(f"Acumulando Distancia {camino[-1]} -> {camino[0]} = {dij}")
    
    return coste

In [8]:
def getCostePivotes(distancias, caminoAnt, costeAnt, piv1, piv2):
      
    caminoAnt1 = [caminoAnt[piv1-1], caminoAnt[piv1]]
    if(piv1 != len(distancias)-1):
        caminoAnt1.append(caminoAnt[piv1+1])
    else:
        # Pivote es el último elemento: último al primero
        caminoAnt1.append(caminoAnt[0])
        
    caminoAnt2 = [caminoAnt[piv2-1], caminoAnt[piv2]]
    if(piv2 != len(distancias)-1):
        caminoAnt2.append(caminoAnt[piv2+1])
    else:
        # Pivote es el último elemento: último al primero
        caminoAnt2.append(caminoAnt[0])
        
    caminoNuevo1 = caminoAnt1.copy()
    if caminoAnt[piv2] in caminoNuevo1:
        caminoNuevo1[caminoNuevo1.index(caminoAnt[piv2])] = caminoAnt[piv1]
    caminoNuevo1[1] = caminoAnt[piv2]
    
    
    caminoNuevo2 = caminoAnt2.copy()
    if caminoAnt[piv1] in caminoNuevo2:
        caminoNuevo2[caminoNuevo2.index(caminoAnt[piv1])] = caminoAnt[piv2]
    caminoNuevo2[1] = caminoAnt[piv1]
   
    #print("SubCaminos anteriores: ", caminoAnt1, caminoAnt2)
    #print("SubCaminos nuevos: ", caminoNuevo1, caminoNuevo2)
    
    return (costeAnt - distancias[caminoAnt1[0]][caminoAnt1[1]] - distancias[caminoAnt1[1]][caminoAnt1[2]] - distancias[caminoAnt2[0]][caminoAnt2[1]] - distancias[caminoAnt2[1]][caminoAnt2[2]] + distancias[caminoNuevo1[0]][caminoNuevo1[1]] +  distancias[caminoNuevo1[1]][caminoNuevo1[2]] + distancias[caminoNuevo2[0]][caminoNuevo2[1]] + distancias[caminoNuevo2[1]][caminoNuevo2[2]])
    

In [9]:
# Algoritmo Greedy
def algoritmoGreedy(distancias):
    
    caminoSolucion = [0] # Empezamos en el primer indice de nodo
    nodosPendientes = list(range(1,len(distancias[1]))) # Lista del segundo al ultimo indice de nodo
    
    while len(nodosPendientes) > 0: # Mientras queden nodos por visitar

        nodoActual = caminoSolucion[-1] # Partimos desde el útlimo nodo añadido
        minDistancia = distancias[nodoActual][nodosPendientes[0]] # Inicializamos la distancia del actual al primer pendiente
        nodoMinDistancia = nodosPendientes[0]
        
        for nodoVecino in nodosPendientes: # Recorremos todos los nodos pendientes y nos quedamos con el de menor distancia
            d = distancias[nodoActual][nodoVecino]
            
            if d < minDistancia:
                minDistancia = d
                nodoMinDistancia = nodoVecino
                
        caminoSolucion.append(nodoMinDistancia)
        nodosPendientes.remove(nodoMinDistancia)
        
    return caminoSolucion
    #return [indice+1 for indice in caminoSolucion] # + 1 a cada elemento (tiene que empezar por 1)

In [10]:
def plt_dynamic(fig, x, y, ax, colors=['b']):
    for color in colors:
        ax.plot(x, y, color)
    fig.canvas.draw()


# Estructura de datos

In [2]:
class Hormiga:
    
    # Métodos de clase
    def set_nodos_distancias(num_nodos, distancias):
        Hormiga.num_nodos = num_nodos
        Hormiga.distancias = distancias
    
    # Métodos de instancias
    
    # Operador minimo
    # https://stackoverflow.com/questions/3621826/python-minimum-of-a-list-of-instance-variables
    def __lt__(self, other):
        return self.coste < other.coste
    
    def __eq__(self, other):
        return self.camino == other.camino
    
    def __str__(self):
        return str(self.coste) 
    
    def __repr__(self):
        return str(self.coste)
    
    def __init__(self, *args):
        
        if not Hormiga.num_nodos or not Hormiga.distancias:
            raise TypeError("No se ha inicializado el num nodos o las distancias para la clase Hormiga") 
        
        if len(args) == 0 or len(args) == 1 and args[0]==None:
            # Hormiga con nodo inicial aleatorio
            self.camino = [random.randint(0,Hormiga.num_nodos-1)]
            self.coste = 0
            
        elif len(args) == 1:
            # Hormiga con nodo inicial
            self.camino = [args[0]]
            self.coste = 0
            
        elif len(args) == 2:
            self.camino = args[0]
            self.coste = args[1]
        
        else:
            raise TypeError("Número de argumentos no válido, solo 0 ó 2") 
            
        self.nodos_visitados =  [False for i in range(Hormiga.num_nodos)]
        self.arcos_visitados =  [[False] * Hormiga.num_nodos for i in range(Hormiga.num_nodos)]
            
    def add(self, nodo):
        # Marcamos visitada el nodo y el arco
        self.nodos_visitados[nodo] = True
        self.arcos_visitados[self.camino[-1]][nodo] = True
        self.arcos_visitados[nodo][self.camino[-1]] = True
        
        # Añadimos a la lista
        self.coste += self.distancias[self.camino[-1]][nodo]
        self.camino.append(nodo)
    
    def contiene_arco(self, r, s):
        return self.arcos_visitados[r][s]
    
    def contiene_nodo(self, nodo):
        return self.nodos_visitados[nodo]

# Funciones

In [3]:
def actualizacion_feromona(valor_actual, hormigas, evaporacion, r, s):
    
    termino_evap = (1 - evaporacion) * valor_actual
    
    termino_aporte = 0
#     for i in range(len(hormigas)):
#         if contiene_arco(hormigas[i], r,s) or contiene_arco(hormigas[i], s,r):
    for hormiga in hormigas:
        if hormiga.contiene_arco(r,s):
            termino_aporte += 1 / hormigas[i].coste
        # Si no, cero
    
    return termino_evap + termino_aporte
            

In [4]:
def actualizacion_feromonas(feromona, hormigas, evaporacion):
    for i in range(len(hormigas[0].camino)):
            for j in range(len(hormigas[0].camino)):
                termino_evap = (1 - evaporacion) * feromona[i][j]
    
                termino_aporte = 0
#                 for k in range(len(hormigas)):
#                     if contiene_arco(hormigas[k], i,j) or contiene_arco(hormigas[k], j,i):
                for hormiga in hormigas:
                    if hormiga.contiene_arco(i,j):
                        termino_aporte += 1 / hormiga.coste
                
                feromona[i][j] = termino_evap + termino_aporte
    
    return feromona

In [ ]:
def actualizacion_feromona_elite(valor_actual, hormigas, evaporacion, r, s, num_elite):
    
    termino_evap = (1 - evaporacion) * valor_actual
    
    termino_aporte = 0
    for hormiga in hormigas:
        if hormiga.contiene_arco(r,s):
            termino_aporte += 1 / hormiga.coste
        # Si no, cero
        
    # SHE: refuerzo de buenos arcos
    mejor_global = min(hormigas)
    if mejor_global.contiene_arco(r,s):
        termino_aporte += num_elite * 1 / mejor_global.coste
    
    return termino_evap + termino_aporte

In [ ]:
# import bisect
# def cdf(weights):
#     total = sum(weights)
#     result = []
#     cumsum = 0
#     for w in weights:
#         cumsum += w
#         result.append(cumsum / total)
#     return result

# def eligeProbabilidad(population, weights):
#     assert len(population) == len(weights) # Comprobar mismo tamaño
#     cdf_vals = cdf(weights) # Suma acumulativa de los pesos
#     x = random.random() 
#     idx = bisect.bisect(cdf_vals, x) # Índice donde insertar el número aleatorio
#     return population[idx] # Elemento elegido según probabilidad

# def regla_transicion(hormiga, feromona, heuristica, alpha, beta):
    
#     no_visitados = [x for x in list(range(len(hormiga.camino))) if x not in hormiga.camino]      
#     probabilidades = []
    
#     # CALCULO DE PROBABILIDAD DE TRANSICIÓN 
#     # Por cada nodo
#     for s in range(len(hormiga.camino)):  
        
#         # Nodo ya visitado
#         if s in hormiga.camino:
#             probabilidades.append(0)
#         # Nodo no visitado
#         else:
#             num = feromona[hormiga.camino[-1]][s]**alpha * heuristica[hormiga.camino[-1]][s]**beta
#             den = 0.00001 # Para evitar división por cera (último nodo en elegir)
#             # Acumulamos el sumatorio de los nodos no visitados
#             for u in range(len(hormiga.camino)):
#                 if u in no_visitados :
#                     den += feromona[hormiga.camino[-1]][u]**alpha * heuristica[hormiga.camino[-1]][u]**beta
            
#             probabilidades.append(num/den)
        
#     # Elegimos uno de los nodos con la probabilidad calculada
#     return eligeProbabilidad(list(range(len(hormiga.camino))), probabilidades)

In [ ]:
def regla_transicion_SCH(hormiga, feromona, heuristica, alpha, beta, q0):
    
    # Lanzamos el dado q
    q = random.random()
    
    # Sale el dado: nodo más probable
    if q <= q0:
    
        # CALCULO DE PROBABILIDAD DE TRANSICIÓN 
        # Por cada nodo
        probabilidades = []
        for s in range(Hormiga.num_nodos):  

            # Nodo ya visitado
            if s in hormiga.camino:
                probabilidades.append(0)
            # Nodo no visitado
            else:
                probabilidades.append(feromona[hormiga.camino[-1]][s]**alpha * heuristica[hormiga.camino[-1]][s]**beta)
        
        # Devolvemos nodo mas probable
        return probabilidades.index(max(probabilidades))
                
    else: # No sale el dado
        # Un nodo aleatorio
        no_visitados = [x for x in list(range(Hormiga.num_nodos)) if x not in hormiga.camino]      
        return random.choice(no_visitados)
    
        

In [ ]:
def actualizacion_feromona_global(valor_actual, mejor_global, evaporacion):
    
    termino_evap = (1 - evaporacion) * valor_actual
    
    termino_aporte = evaporacion * 1 / mejor_global.coste
    
    return termino_evap + termino_aporte

In [10]:
def actualizacion_feromona_local(valor_actual, feromona_ini, evaporacion):
    termino_evap = (1 - evaporacion) * valor_actual
    
    termino_aporte = evaporacion * feromona_ini
    
    return termino_evap + termino_aporte

1

In [5]:
import bisect
def cdf(weights):
    total = sum(weights)
    result = []
    cumsum = 0
    for w in weights:
        cumsum += w
        result.append(cumsum / total)
    return result

def eligeProbabilidad(population, weights):
    assert len(population) == len(weights) # Comprobar mismo tamaño
    cdf_vals = cdf(weights) # Suma acumulativa de los pesos
    x = random.random() 
    idx = bisect.bisect(cdf_vals, x) # Índice donde insertar el número aleatorio
    return population[idx] # Elemento elegido según probabilidad

def regla_transicion(hormiga, feromona, heuristica, alpha, beta, num_nodos):
    
#     no_visitados = []
#     for x in list(range(num_nodos)):
#         if x not in hormiga.camino:
#             no_visitados.append(x)
            
    probabilidades = []
    
    # CALCULO DE PROBABILIDAD DE TRANSICIÓN 
    # Por cada nodo
    for s in range(num_nodos):  
        
        # Nodo ya visitado
        if hormiga.contiene_nodo(s):
            probabilidades.append(0)
        # Nodo no visitado
        else:
            num = feromona[hormiga.camino[-1]][s]**alpha * heuristica[hormiga.camino[-1]][s]**beta
            den = 0 # Para evitar división por cera (último nodo en elegir)
            # Acumulamos el sumatorio de los nodos no visitados
            for u in range(num_nodos):
                if not hormiga.contiene_nodo(u):
                    den += feromona[hormiga.camino[-1]][u]**alpha * heuristica[hormiga.camino[-1]][u]**beta

            probabilidades.append(num/den if den > 0 else 1)
        
    # Elegimos uno de los nodos con la probabilidad calculada
    return eligeProbabilidad(list(range(num_nodos)), probabilidades)